In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [4]:
traindata = np.loadtxt('traindata.csv', delimiter=',')
trainlabel = np.loadtxt('trainlabel.csv', delimiter=',')
validdata = np.loadtxt('validdata.csv', delimiter=',')
validlabel = np.loadtxt('validlabel.csv', delimiter=',')

In [9]:
def train(traindata, trainlabel, validdata, validlabel, activation, lr, epochs, debug=False):
    neuron = nn.Linear(9, 1)
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    for i in range(1, epochs + 1):
        Z, dZdW, dZdb = neuron.forward(traindata.T)
        L, dLdZ = mse_loss(Z, trainlabel, with_grad=True)
        train_loss.append(L)
        train_acc.append(((Z >= 0.5) == trainlabel).sum() / trainlabel.size)

        dLdW = np.sum(dLdZ * dZdW, axis=1)
        dLdb = np.sum(dLdZ * dZdb, axis=0)

        neuron.weights -= dLdW * lr
        neuron.bias -= dLdb * lr

        val_out, _, _ = neuron.forward(validdata.T, with_grad=False)
        val_loss.append(mse_loss(val_out, validlabel, with_grad=False)[0])
        val_acc.append(((val_out >= 0.5) == validlabel).sum() / validlabel.size)

        if debug:
            print('epoch {}, train loss {:2f} acc {:2f} validation loss {:2f} acc {:2f}'.format(i, L, train_acc[-1], val_loss[-1], val_acc[-1]))
    
    return neuron, train_loss, train_acc, val_loss, val_acc

def plot_history(train_loss, train_acc, val_loss, val_acc):
    plt.subplot(1, 2, 1)
    plt.plot(train_loss)
    plt.plot(val_loss)
    plt.subplot(1, 2, 2)
    plt.plot(train_acc)
    plt.plot(val_acc)

In [11]:
train(traindata, trainlabel, validdata, validlabel, torch.sigmoid, 0.01, 100, debug=True)

TypeError: sigmoid(): argument 'input' (position 1) must be Tensor, not Linear